This notebook reproduces the AAV design experiments whose results are shown in Fig 5.

Variable name suffixes in the following cells denote array dimensions, where

n: number of calibration and test data points  
l: number of values of the inverse temperature, lambda  
L: length of sequence  
t: number of trials of sampling from test distribution, per lambda  
s: number of samples from test distribution  
m: number of calibration data points  
m1: m + 1  

In [1]:
import os
import sys
import time
from importlib import reload
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import scipy as sc
from tensorflow import keras

import assay
import calibrate as cal

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams["font.size"] = 16

import seaborn as sns
sns.set_style('darkgrid')

/home/clarafy/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/clarafy/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/clarafy/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/clarafy/anaconda

## Load held-out data and parameters of test sequence distributions

In [3]:
# load held-out data (calibration and test data)
d = np.load('../aav/test_and_calibration_data.npz')
seq_n = d['seq_n']  # list of strings
y_n = d['y_n']      # true fitnesses
n = y_n.size
print('Loaded {} held-out test and calibration data points.'.format(n))

Loaded 1000000 held-out test and calibration data points.


In [4]:
# load parameters of test sequence distributions
d = np.load('../aav/models/constrained_maxent.npz')

phitestnuc_lxLxk = d['phitestnuc_lxLxk']
# phitestnuc_lxLxk[i] is an (L, k) numpy array of unnormalized probabilities of a categorical distribution
# over k = 4 nucleotides at each of L sequence positions,
# corresponding to phi in Eq. 5 of Supp. Materials and Methods here:
# https://www.biorxiv.org/content/10.1101/2021.11.02.467003v2.full

lambda_l = 1 / d['temperature_l']
# note that lambda in bioRxiv above corresponds to 1 / lambda for us

meanpredfit_l = d['meanpredfit_l']

## Construct confidence sets for designed sequences

Compute predictions and scores for all held-out data (calibration and test data).

In [5]:
n_model = 3
datagen = assay.DataGenerator(seq_n)
pred_n = np.zeros([n])

# load ensemble of trained NNs and take their average prediction for all held-out sequences
for seed in range(n_model):
    model = keras.models.load_model('../aav/models/h100_seed{}.npy'.format(seed))
    pred_n += model.predict_generator(datagen).reshape(n)
pred_n /= n_model
score_n = np.abs(pred_n - y_n)  # score with residual

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/clarafy/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/layers/core.py:895: UserWarning: aav.modeling is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Use rejection sampling to sample from test distribution, and construct confidence sets for resulting designed sequences.

In [ ]:
n_trial = 100
alpha = 0.1
n_cal = 10000

# compute training likelihoods of all sequences
logptrain_n = assay.get_loglikelihood(seq_n, assay.PNNKAA_LXK)

n_lambda = phitestnuc_lxLxk.shape[0]
cov_lxt = np.zeros([n_lambda, n_trial])
avglen_lxt = np.zeros([n_lambda, n_trial])
len_lxt = {(l, t): None for l, t in zip(range(n_lambda), range(n_trial))}
fit_lxt = {(l, t): None for l, t in zip(range(n_lambda), range(n_trial))}

for l in range(n_lambda):
    t0 = time.time()
    print("Test distribution with lambda = {:.1f}".format(lambda_l[l]))
    
    # compute acceptance probabilities for all sequences (for rejection sampling from test distribution)
    paccept_n, logptest_n = assay.get_rejection_sampling_acceptance_probabilities(
        seq_n, phitestnuc_lxLxk[l], logptrain_n)

    # compute (unnormalized) weights for all data
    w_n = np.exp(logptest_n - logptrain_n)

    for t in range(n_trial):
        
        # partition held-out data into calibration data and test data
        # (i.e., samples from proposal distribution for rejection sampling from test distribution)
        shuffle_idx = np.random.permutation(n)
        cal_idx, test_idx = shuffle_idx[: n_cal], shuffle_idx[n_cal :]
        
        # sample from test distribution using rejection sampling
        testsamp_idx = assay.rejection_sample_from_test_distribution(paccept_n[test_idx])
        n_test = testsamp_idx.size
        if t == 0:
            print("  On trial 0, sampled {} sequences from the test distribution.".format(n_test))

        # fetch and normalize weights of calibration data
        p_sxm1 = np.hstack([np.tile(w_n[cal_idx], [n_test, 1]), w_n[test_idx[testsamp_idx]][:, None]])
        p_sxm1 /= np.sum(p_sxm1, axis=1, keepdims=True)
        
        # compute quantile of weighted calibration scores
        augscore_sxm1 = np.tile(np.hstack([score_n[cal_idx], [np.infty]]), (n_test, 1))
        q_sx1 = cal.get_weighted_quantile(1 - alpha, p_sxm1.T, augscore_sxm1.T)[:, None]
        
        # construct confidence intervals
        testpred_sx1 = pred_n[test_idx[testsamp_idx]][:, None]
        lu_sx2 =  np.hstack([testpred_sx1 - q_sx1, testpred_sx1 + q_sx1])
         
        # record confidence interval lengths, true fitnesses, and empirical coverage
        avglen_lxt[l, t] = np.mean(2 * q_sx1)
        len_lxt[(l, t)] = 2 * q_sx1.flatten()
        fit_lxt[(l, t)] = y_n[test_idx[testsamp_idx]]
        cov_lxt[l, t] = cal.get_split_coverage(lu_sx2, fit_lxt[(l, t)])
        
    print("  Empirical coverage{:.4f}, average length {:.2f} ({:.1f} s)".format(
        np.mean(cov_lxt[l]), np.mean(avglen_lxt[l]), time.time() - t0))
    
    # save after each lambda
    np.savez('../aav/split-results.npz',
        cov_lxt=cov_lxt, avglen_lxt=avglen_lxt, len_lxt=len_lxt, fit_lxt=fit_lxt,
    )

Test distribution with lambda = 1.0
  On trial 0, sampled 8646 sequences from the test distribution.
